In [2]:
import cv2
import numpy as np
import pandas as pd
import re
import os
from paddleocr import PaddleOCR
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings

warnings.filterwarnings('ignore') #Ignore all Python warnings / 忽略所有Python警告

print("All libraries imported successfully!")
print("所有库导入成功！")

All libraries imported successfully!
所有库导入成功！


# PART 1: Image Preprocessing Functions
# 第一部分：图像预处理函数

In [3]:
# Load image from file / 从文件加载图像
def load_image(image_path):
    if not os.path.exists(image_path):
        print(f"❌ Error: File not found - {image_path}")
        return None
    
    image = cv2.imread(image_path)
    if image is None:
        print("❌ Error: Cannot read image")
        return None
    
    return image

# Complete image preprocessing pipeline / 完整的图像预处理流程
def preprocess_image(image_path):

    # Load image / 加载图像
    original = load_image(image_path)
    if original is None:
        return None
    
    # Convert to grayscale / 转换为灰度图
    gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    
    # Remove noise / 去除噪声
    denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)
    
    # Apply threshold / 应用阈值
    threshold = cv2.adaptiveThreshold(
        denoised, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )
    
    return threshold


# PART 2: OCR Recognition Functions
# 第二部分：OCR识别函数

In [4]:
#Initialize PaddleOCR engine / 初始化PaddleOCR引擎
def initialize_ocr():
    ocr = PaddleOCR(use_angle_cls=True, lang='en')
    return ocr

#Perform OCR on image / 对图像执行OCR
def perform_ocr(ocr, image_path):
    #print("Get all text from OCR... / 执行PCR获取原始数据")
    result = ocr.ocr(image_path)
    
    if result is None or len(result) == 0:
        print("❌ OCR returned no results!")
        return []
    
    #print("="*70 + "\n")
    #打印ocr返回的初始结果
    #print(result)
    #print("="*70 + "\n")
    return result[0] if len(result) > 0 else []

#Organize OCR results / 整理OCR结果
def organize_ocr_results(ocr_result):
    #organized_list = []
    df_ocr_result = pd.DataFrame(columns=["text","confidence","left_top_x","left_top_y","right_top_x","right_top_y","left_bottom_x","left_bottom_y","right_bottom_x","right_bottom_y","center_x","center_y"])
    
    print("ORGANIZING OCR DATA / 整理OCR数据:")
    print("="*70)

    texts = ocr_result.get('rec_texts', [])
    scores = ocr_result.get('rec_scores', [1.0] * len(texts))
    polys = ocr_result.get('rec_polys', [])

    
    for idx, text in enumerate(texts):
        confidence = scores[idx] if idx < len(scores) else 1.0   #当scores和text长度不一致用1.0补全
        
        # Get coordinates / 获取多坐标
        if idx < len(polys) and len(polys[idx]) >= 4:
            poly = polys[idx]
            
            # Calculate center point / 计算中心点
            center_x = (poly[0][0] + poly[2][0]) / 2
            center_y = (poly[0][1] + poly[2][1]) / 2
            
            # Format: (x1,y1)(x2,y2)(x3,y3)(x4,y4)(center_x,center_y)
            pos_str = f"({poly[0][0]:.0f},{poly[0][1]:.0f})({poly[1][0]:.0f},{poly[1][1]:.0f})({poly[2][0]:.0f},{poly[2][1]:.0f})({poly[3][0]:.0f},{poly[3][1]:.0f})({center_x:.0f},{center_y:.0f})"
            
        else:
            pos_str = "(0,0)(0,0)(0,0)(0,0)(0,0)"
            center_x = 0
            center_y = 0
            y_position = idx * 10
            x_position = 0
        
        # Print in compact format / 以紧凑格式打印
        #print(f"{idx+1}. Text: '{text}' | Confidence: {confidence:.2f} | Position: {pos_str}")

        # Save text into dataframe / 把结果保存到dataframe
        df_ocr_result.loc[len(df_ocr_result.index)] = [str(text),float(confidence),poly[0][0],poly[0][1],poly[1][0],poly[1][1],poly[2][0],poly[2][1],poly[3][0],poly[3][1],center_x,center_y]
        
    #print("\n" + "="*70 + "\n")
    #organized_list.sort(key=lambda x: x['left_top_p'][1])
    #print("\n" + "="*70 + "\n")
    return df_ocr_result

# PART 3: Information Extraction Functions
# 第三部分：信息提取函数

In [5]:
#These functions must be called in order / 这部分的函数必须按顺序调用

#Extract date from text / 从文本中提取日期
def extract_date(text):
    patterns = [
        r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}',        # 25/12/2018 or 12-25-18
        r'\d{4}[-/]\d{1,2}[-/]\d{1,2}',          # 2018/12/25
        r'\d{1,2}\.\d{1,2}\.\d{2,4}',            # 25.12.2018
        r'\d{1,2}\s+\w+\s+\d{2,4}',              # 25 Dec 2018
        r'\w+\s+\d{1,2},?\s+\d{2,4}',            # Dec 25, 2018
        r'\d{1,2}(st|nd|rd|th)?\s+\w+\s+\d{4}',  # 25th December 2018
    ]
 
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            found_date = match.group()
            #print(f"✅ Found date: '{found_date}' in text: '{text}'")
            #print("="*70 + "\n")
            return found_date
    return 0

#====================================================================================================
# Classify text type / 分类文字类型
def classify_text_type(text):
    """
    Classify text type / 分类文字类型
    
    Returns:
    - 'price' : Price (two decimal places) / 价格（两位小数）
    - 'code' : Code (long digit string or alphanumeric) / 编码（长数字串或字母数字混合）
    - 'header' : Header keyword (PRICE, QTY, AMOUNT, etc.) / 表头关键字（PRICE, QTY, AMOUNT 等）
    - 'payment' : Payment keyword (TOTAL, CASH, CHANGE, etc.) / 结账关键字（TOTAL, CASH, CHANGE 等）
    - 'text' : Regular text / 普通文字
    """  
    
    text_clean = text.strip()
    text_lower = text_clean.lower()


    # 1. Check for date type / 检查是否是日期
    if extract_date(text) != 0:
        return 'date'
    
    
    # 2. Check for header keywords / 检查表头关键字
    header_keywords = ['price', 'amount', 'qty', 'quantity', 'item', 
                      'description', 'product', 'code', 'desc']
    
    if any(kw == text_lower or kw in text_lower for kw in header_keywords):
        if len(text_clean) < 20:  # Header keywords are usually short
            return 'header'
    
    # 3. Check for payment keywords / 检查结账关键字
    payment_keywords = ['total', 'subtotal', 'grand total', 
                       'cash', 'payment', 'paid',
                       'change', 'balance',
                       'tax', 'vat', 'gst',
                       'discount', 'promo']
    
    if any(kw in text_lower for kw in payment_keywords):
        return 'payment'
    
    # 4. Check if it's a price (number with 2 decimals) / 检查是否是价格
    # Pattern: $12.50 or 12.50 or 12.50$
    price_pattern = r'^\$?\s*\d+[.,]\d{2}\s*\$?$'
    if re.match(price_pattern, text_clean):
        return 'price'
    
    # 5. Check if it is a product code / 检查是否是商品编码
    text_no_space = text_clean.replace(' ', '').replace('-', '').replace('.', '')
    
    if len(text_no_space) > 0:
        # Rule 1: Pure digits with length > 3 (4 or more digits)
        # 规则1：纯数字且长度大于3（4位及以上）
        if text_no_space.isdigit() and len(text_no_space) > 3:
            return 'code'
        
        # Rule 2: Digit ratio > 70%
        # 规则2：数字占比大于70%
        digit_count = sum(c.isdigit() for c in text_no_space)
        total_length = len(text_no_space)
        digit_ratio = digit_count / total_length
        
        # Also require minimum length > 5 to avoid short mixed text
        # 同时要求长度>3，避免短文本被误判
        if digit_ratio > 0.7 and total_length > 3:
            return 'code'

     # 6. Check for symbols / 检查是否是符号 
    if len(text_no_space) > 0:
        # Remove all letters and digits / 移除所有字母和数字
        text_no_alnum = re.sub(r'[a-zA-Z0-9]', '', text_no_space)
        
        # Calculate symbol ratio / 计算符号占比
        symbol_ratio = len(text_no_alnum) / len(text_no_space)
        
        # If 80% or more are symbols, mark as invalid
        # 如果80%以上都是符号，标记为invalid
        if symbol_ratio >= 0.8:
            return 'invalid'

        if len(text_no_space) < 4:
            return 'invalid'

    
    # 7. Otherwise it's text / 其他情况是普通文字
    return 'text'

#====================================================================================================
# Add row number column / 增加行号列
def mark_row_number(df):
    # Add row_No column / 添加 row_No 列
    print("ADDING ROW NUMBER COLUMN / 添加行号:")
    print("="*70)
    
    row_no = 1  # Start from row 1 / 从第1行开始
    df.loc[0, 'row_No'] = row_no  # First element corresponds to row 1 / 第一个元素是第1行
    
    # Iterate through each row and check if it's a new row / 遍历每一行，判断是否为新行
    for i in range(1, len(df)):
        # Current row's center_y / 当前行的 center_y
        current_y = df.loc[i, 'left_bottom_y']
        
        # Previous row's center_y / 前一行的 center_y
        previous_y = df.loc[i-1, 'left_bottom_y']
        
        # Calculate the difference / 计算差距
        diff = abs(current_y - previous_y)
        
        # If difference < 20, consider same row; otherwise, assign a new row_No
        # 如果差距小于20，认为是同一行，否则新增一个行号
        if diff < 20:
            df.loc[i, 'row_No'] = row_no
        else:
            row_no += 1  # For a new row, increment row_No by 1 / 新的一行，row_No 加 1
            df.loc[i, 'row_No'] = row_no
    
    # Convert to integer type / 转换为整数类型
    df['row_No'] = df['row_No'].astype(int)
    return df

#====================================================================================================
# Add priority column / 添加priority列
def add_priority_column(df):
    """
    Rule:
    - Find first row where mark='header' → row_head
    - Find first row where text contains 'total' → row_tail
    - Rows where row_head <= row_No < row_tail → mark as 1
    - Other rows → mark as 0
    
    规则：
    - 找到mark为'header'的第一行 → row_head
    - 找到text包含'total'的第一行 → row_tail
    - row_head <= row_No < row_tail 的行 → 标记为1
    - 其他行 → 标记为0
    """

    print("ADDING PRIORITY COLUMN / 添加有效信息权重列:")
    print("="*70)
    
    # Initialize / 初始化
    df['priority'] = 0
    
    # Find row_head (first header row) / 找到row_head（第一个header行）
    header_rows = df[df['mark'] == 'header']
    
    if len(header_rows) > 0:
        row_head = header_rows['row_No'].min()
        #print(f"✅ Found header at row_No: {row_head}")
        #print(f"  Header texts: {list(header_rows['text'].values)}")
    else:
        #print("❌ No header found, using row_No 0 as row_head")
        row_head = 0
    
    # Find row_tail (first row with 'total' in text) / 找到row_tail
    total_rows = df[df['text'].str.lower().str.contains('total', na=False)]
    
    if len(total_rows) > 0:
        row_tail = total_rows['row_No'].min()
        #print(f"✅ Found 'total' at row_No: {row_tail}")
        #print(f"  Total text: '{total_rows.iloc[0]['text']}'")
    else:
        #print("❌ No 'total' found, using last row as row_tail")
        row_tail = df['row_No'].max() + 1
    
    # Mark valid rows / 标记有效行
    # Rows where row_head <= row_No < row_tail → mark as 1
    # Other rows → mark as 0
    df['priority'] = ((df['row_No'] >= row_head) & (df['row_No'] < row_tail)).astype(int)
    
    # # Count valid rows / 统计有效行
    # valid_count = df['priority'].sum()
    
    # print(f"\nValid data range / 有效数据范围:")
    # print(f"  From row_No {row_head} to row_No {row_tail-1} (inclusive)")
    
    # print(f"\nResults / 结果:")
    # print(f"  Valid rows (priority=1): {valid_count}")
    # print(f"  Invalid rows (priority=0): {len(df) - valid_count}")
    
    # print("="*70 + "\n")
    
    return df
    
#====================================================================================================    
#Add column number column / 增加列号列
def add_column_number(df):
    """
    Add col_No column based on x position clustering
    根据x位置聚类添加col_No列
    
    Rule: If any x coordinate (left_top_x, right_top_x, or center_x) 
          differs by <= 20, they are in the same column
    规则：如果任一x坐标相差<=20，就是同一列
    """
    
    print("ADDING COLUMN NUMBERS / 添加列号:")
    print("="*70)
    
    # Initialize / 初始化
    df['col_No'] = 0
    
    
    # Sort by left_top_x (left to right), then by left_top_y (top to bottom) 
    # 先按 left_top_x 排序（从左到右），再按 left_top_y 排序（从上到下）
    df_sorted = df.sort_values(['left_top_x', 'left_top_y']).reset_index(drop=True)

    
    columns = []  # Store coordinates of all elements in the current column / 用于存储当前列所有元素的坐标
    curr_col_No = 1

    for row in df_sorted.itertuples():
        left_top_x = row.left_top_x
        right_top_x = row.right_top_x
        width = right_top_x - left_top_x
        text = row.text

        for item in columns:
            #left_top_x -> right_top_x forms a line segment / left_top_x -> right_top_x 形成一条线段 
            #Check if the two segments intersect / 判断两者是否交集
            is_intersect = item[0] < right_top_x and item[1] > left_top_x
            
            if is_intersect:
                # Check if overlap ratio > 70% for either side; if so, consider as same column 
                # 判断双方的交集比例是否有一方超过70%，有则认为是同一列
                intersect_len = min(item[1], right_top_x) - max(item[0], left_top_x)
                is_intersect = intersect_len / width > 0.7 or intersect_len / (item[1] - item[0]) > 0.7
                # print(f"{intersect_len / width} {intersect_len / (item[1] - item[0])}")

            if not is_intersect:
                # Switch to new column and clear / 切换到新列并清空
                columns.clear()
                curr_col_No += 1
                # print(f"New column {curr_col_No} created")
                # print(f"{left_top_x} {right_top_x} {item[0]} {item[1]} {item[2]} {row.text}")
                break

        columns.append((left_top_x, right_top_x, row.text))
        # Assign column number / 分配列号
        df_sorted.loc[row.Index, 'col_No'] = curr_col_No

    
    # Copy col_No back to original dataframe order
    # 将col_No复制回原始dataframe的顺序
    # df['col_No'] = df_sorted['col_No'].values
    df = df.drop(columns=['col_No'])  # drop original
    df = df.merge(
        df_sorted[['left_top_x', 'left_top_y', 'col_No']],
        on=['left_top_x', 'left_top_y'],
        how='left'
    )
    
    df['col_No'] = df['col_No'].astype(int)
    
    #print("\n" + "="*70)
    #print(f"识别到的列数: {curr_col_No}")    
    #print("="*70 + "\n")
    
    return df
    
#====================================================================================================
# Update category text type: mark as "Unit" / 更新分类文字类型：标记为“单位”
def mark_units(df):
    """
    Mark units directly in the 'mark' column
    直接在mark列中标记单位

    Rules::
    1. Directly matches common unit labels
    2. Number followed by common unit labels
    3. Pure number in the same column as header "Qty"

    规则：
    1. 直接显示为常见的单位标识
    2.数字+常见单位标识
    3.与表头 “Qty” 同列的纯数字
    """
    
    print("MARKING UNITS / 标记单位:")
    print("="*70)
    
    # Common units / 常见单位
    unit_keywords = [
        # Counting / 计数
        'PC', 'PCS', 'PIECE', 'PIECES', 'EA', 'EACH', 'UNIT',
        
        # Weight / 重量
        'KG', 'G', 'GM', 'GRAM', 'LB', 'POUND', 'OZ', 'OUNCE',
        
        # Volume / 体积
        'L', 'ML', 'LITER', 'GAL', 'GALLON', 'FL OZ',
        
        # Packaging / 包装
        'BOX', 'BAG', 'BTL', 'BOTTLE', 'CAN', 'PKT', 'PACKET',
        'CTN', 'CARTON', 'JAR', 'TIN', 'TUBE',
        
        # Other / 其他
        'PAIR', 'SET', 'ROLL', 'PACK', 'BUNCH', 'DOZEN', 'DZ'
    ]
    
    # Find Qty column if exists / 查找Qty列
    qty_headers = df[(df['mark'] == 'header') & 
                    (df['text'].str.upper().str.contains('QTY|QUANTITY', na=False))]
    
    qty_col = None
    if len(qty_headers) > 0:
        qty_col = qty_headers.iloc[0]['col_No']
        #print(f"Found Qty column at col_No: {qty_col}")
        #print(f"找到Qty列，列号: {qty_col}\n")
    
    count = 0
    
    # Check each row / 检查每一行
    for idx in df.index:
        text = df.loc[idx, 'text']
        text_upper = text.upper().strip()
        current_mark = df.loc[idx, 'mark']
        
        is_unit = False
        reason = ""
        
        # Rule 1: Keyword match / 规则1：关键词匹配
        if text_upper in unit_keywords:
            is_unit = True
            reason = f"keyword match: {text_upper}"
        
        # Rule 2: Pattern match (e.g., "1 PC") / 规则2：模式匹配
        if not is_unit:
            pattern = r'^\d+\.?\d*\s*(' + '|'.join(unit_keywords) + r')$'
            if re.match(pattern, text_upper):
                is_unit = True
                reason = f"pattern match: {text}"
        
        # Rule 3: Pure number in Qty column / 规则3：Qty列的纯数字
        if not is_unit and qty_col is not None:
            current_col = df.loc[idx, 'col_No']
            
            if current_col == qty_col:
                text_clean = text.replace('.', '').replace(',', '')
                if text_clean.isdigit():
                    try:
                        num = float(text)
                        if 0 < num < 1000:
                            is_unit = True
                            reason = f"number in Qty column: {text}"
                    except:
                        pass
        
        # Update mark to 'unit' if identified / 如果识别为单位，更新mark为'unit'
        if is_unit:
            df.loc[idx, 'mark'] = 'unit'
            count += 1
            
            row_no = df.loc[idx, 'row_No']
            col_no = df.loc[idx, 'col_No']
            
            #print(f"✅ '{text}' (row={row_no}, col={col_no}) → mark='unit'")
            #print(f"    Reason: {reason}")
            #print(f"    Original mark: {current_mark}")
    
    #print(f"\n✅ Total units marked: {count}")
    #print(f"✅ 标记的单位总数: {count}")
    #print("="*70 + "\n")
    
    return df

#====================================================================================================
# Update priority (mark as 2) / 更新权重（标记为2）
def update_priority_marks(df):
    """
    Update priority marks with additional rules
    用额外规则更新priority标记

    Rules:
    1. If code and text are in same row or adjacent rows in same column，increase text's priority to 2
    1: If a code and text are adjacent in the same row or column, increase text's priority to 2
    3. If text is in same column as 'item' header，increase text's priority to 2

    规则：
    1. 如果存在code和text在同一行或同一列临近行，增加text的 priority 为2
    2. 如果price和包含amount/total/sum等关键词的text在同一列，，增加text的 priority 为2
    3. 如果text和header为'item'在同一列，增加text的 priority 为2
    """
    
    print("UPDATING PRIORITY MARKS / 更新优先级标记:")
    print("="*70)
    
    # Keywords that indicate summary/total amounts
    # 标识总额的关键词
    total_keywords = [
        'amount', 'sum', 'balance', 'due', 'payable', 'net', 'gross'
    ]
    
    # Rule 1: Code + Text in same/adjacent rows in same column / 规则1：同一行或同一列临近行的code和text 
    #print("\nRule 1: Checking code + text pairs / 检查code和text配对:")
    #print("-"*70)
    
    count_rule1 = 0
    
    for idx, row in df.iterrows():
        if row['mark'] == 'text' and row['priority'] == 1:
            text_row = row['row_No']
            text_col = row['col_No']
            
            for idx2, row2 in df.iterrows():
                if row2['mark'] == 'code':
                    code_row = row2['row_No']
                    code_col = row2['col_No']
                    
                    same_row = (text_row == code_row)
                    same_col = (text_col == code_col)
                    adjacent_row = abs(text_row - code_row) == 1
                    
                    if same_row or (same_col and adjacent_row):
                        df.loc[idx, 'priority'] = 2
                        count_rule1 += 1
                        # print(f"✅ Text '{row['text'][:30]}' (row={text_row}, col={text_col}) → priority=2")
                        # print(f"    Paired with code '{row2['text']}' (row={code_row}, col={code_col})")
                        break
    
    # print(f"\nRule 1 applied to {count_rule1} text items")
    
    # Rule 2: Price + Total keywords /规则2：价格和总额关键词   
    # print("Rule 2: Checking price + total keyword pairs / 检查price和总额关键词配对...")
    # print("-"*70)
    
    count_rule2 = 0
    
    for idx, row in df.iterrows():
        if row['mark'] == 'price':
            price_col = row['col_No']
            price_row = row['row_No']
            
            for idx2, row2 in df.iterrows():
                text_lower = row2['text'].lower()
                
                contains_keyword = any(kw in text_lower for kw in total_keywords)
                
                if contains_keyword and row2['col_No'] == price_col:
                    matched_keyword = next((kw for kw in total_keywords if kw in text_lower), '')
                    
                    df.loc[idx, 'priority'] = 2
                    count_rule2 += 1
                    
                    #print(f"✅ Price '{row['text']}' (row={price_row}, col={price_col}) → priority=2")
                    #print(f"    Same column as '{row2['text']}' (contains '{matched_keyword}')")
                    break
    
    #print(f"\nRule 2 applied to {count_rule2} price items")

    
    # Rule 3: Text in same column as 'item' header / 规则3：text和'item'表头在同一列
    
    #print("\n" + "-"*70)
    #print("Rule 3: Checking text in 'item' column / 检查item列的text:")
    #print("-"*70)
    
    count_rule3 = 0
    
    # Find 'item' header column / 查找'item'表头列
    item_headers = df[(df['mark'] == 'header') & 
                     (df['text'].str.upper().str.contains('ITEM|DESCRIPTION|DESC|PRODUCT|NAME', na=False))]
    
    if len(item_headers) > 0:
        item_col = item_headers.iloc[0]['col_No']
        item_header_text = item_headers.iloc[0]['text']
        #print(f"Found item header: '{item_header_text}' at col_No={item_col}\n")
        
        # Mark all text items in this column as priority=2
        # 将该列的所有text项标记为priority=2
        for idx, row in df.iterrows():
            if row['mark'] == 'text' and row['col_No'] == item_col and row['priority'] == 1:
                df.loc[idx, 'priority'] = 2
                count_rule3 += 1
                
                text_row = row['row_No']
                print(f"  ✓ Text '{row['text'][:30]}' (row={text_row}, col={item_col}) → priority=2")
                print(f"    In same column as '{item_header_text}' header")
    else:
        print("❌No 'item' header found / 未找到'item'表头")
    
    #print(f"\nRule 3 applied to {count_rule3} text items")
    
    # #========== Final Summary / 最终总结 ==========
    # print("\n" + "="*70)
    # print("FINAL SUMMARY / 最终总结:")
    # print("="*70)
    
    # priority_counts = df['priority'].value_counts().sort_index()
    
    # for val in [0, 1, 2]:
    #     count = priority_counts.get(val, 0)
    #     if val == 0:
    #         print(f"  priority = 0 (Invalid / 无效): {count} items")
    #     elif val == 1:
    #         print(f"  priority = 1 (Normal valid / 普通有效): {count} items")
    #     elif val == 2:
    #         print(f"  priority = 2 (High priority / 高优先级): {count} items")
    
    # total_changes = count_rule1 + count_rule2 + count_rule3
    # print(f"\nTotal items marked as priority=2: {total_changes}")
    # print(f"标记为priority=2的总项数: {total_changes}")
    
    # print("="*70 + "\n")
    
    return df

#====================================================================================================
def calculate_row_diff(df):
    """
    Calculate row difference between price and text
    计算price和text之间的行差
    
    Logic:
    1. Count the number of price and text entries to find unmatched extra rows
    2. For the type with more entries, calculate row difference to nearest row of the other type
    3. Store row differences in row_diff column
    
    逻辑：
    1. 统计price和text数量，得到不能匹配的多余行数
    2. 对数量多的类型，计算最近行的另一种类型的行差值
    3. 将行差存到row_diff列
    """
    
    print("CALCULATING ROW DIFFERENCES / 计算行差:")
    print("="*70)
    
    # Filter priority=2 and mark in ['text', 'price'] 
    # 筛选priority=2且mark为text或price
    filtered = df[(df['priority'] == 2) & 
                  (df['mark'].isin(['text', 'price']))]
    
    # Count text and price / 统计text和price数量
    text_count = len(filtered[filtered['mark'] == 'text'])
    price_count = len(filtered[filtered['mark'] == 'price'])
    
    #print(f"Text count: {text_count}")
    #print(f"Price count: {price_count}")
    
    # Initialize row_diff column / 初始化row_diff列
    filtered['row_diff'] = None
    
    # Determine which is more / 判断哪个数量多
    need_to_delete = 0
    
    if price_count > text_count:
        #print(f"\n✓ Price is more numerous / Price数量更多")
        #print(f"For each price, finding nearest text / 对每个price查找最近的text\n")
        
        search_type = 'price'
        target_type = 'text'
        need_to_delete = price_count - text_count
        
    elif text_count > price_count:
        #print(f"\n✓ Text is more numerous / Text数量更多")
        #print(f"For each text, finding nearest price / 对每个text查找最近的price\n")
        
        search_type = 'text'
        target_type = 'price'
        need_to_delete = text_count - price_count
    else:
        #print(f"\n❌  Equal count, defaulting to search from price / 数量相等，默认从price搜索")
        search_type = 'price'
        target_type = 'text'
    
    # Process each item / 处理每个条目
    count = 0
    
    for idx in filtered.index:
        # Check if this is the search type
        if filtered.loc[idx, 'priority'] == 2 and filtered.loc[idx, 'mark'] == search_type:
            current_row = filtered.loc[idx, 'row_No']
            current_text = filtered.loc[idx, 'text']
            
            # Find nearest target type / 查找最近的目标类型
            min_diff = float('inf')
            nearest_target = None
            
            for idx2 in filtered.index:
                if filtered.loc[idx2, 'priority'] == 2 and filtered.loc[idx2, 'mark'] == target_type:
                    target_row = filtered.loc[idx2, 'row_No']
                    diff = abs(target_row - current_row)
                    
                    # Update if closer / 如果更近则更新
                    if diff < min_diff:
                        min_diff = diff
                        nearest_target = {
                            'idx': idx2,
                            'row': target_row,
                            'text': filtered.loc[idx2, 'text']
                        }
            
            # Store the difference / 存储差值
            if nearest_target:
                # Calculate signed difference / 计算带符号的差值
                actual_diff = nearest_target['row'] - current_row
                filtered.loc[idx, 'row_diff'] = actual_diff
                count += 1
                
                if actual_diff == 0:
                    direction = "same row / 同一行"
                elif actual_diff > 0:
                    direction = f"{actual_diff} rows below / 下方{actual_diff}行"
                else:
                    direction = f"{abs(actual_diff)} rows above / 上方{abs(actual_diff)}行"
                
                #print(f"  {search_type.upper()} '{current_text}' (row={current_row})")
                #print(f"    → Nearest {target_type}: '{nearest_target['text']}' (row={nearest_target['row']})")
                #print(f"    → row_diff = {actual_diff} ({direction})\n")
            else:
                filtered.loc[idx, 'row_diff'] = None
                #print(f"  {search_type.upper()} '{current_text}' (row={current_row})")
                #print(f"    → No {target_type} found\n")
    
    #print("="*70)
    #print(f"✅ Calculated row_diff for {count} items")
    #print(f"✅ 为{count}项计算了row_diff")
    #print("="*70 + "\n")
    
    return filtered, need_to_delete
#====================================================================================================
def delete_by_row_diff(df, need_to_delete):
    """
    Delete extra rows based on row_diff distribution
    根据 row_diff 分布删除多余行
    
    Logic:
    1. Count row_diff value distribution
    2. Delete rows with least frequent and largest absolute difference
    3. Repeat until deleted need_to_delete rows
    
    逻辑：
    1. 统计row_diff值的分布
    2. 删除数量最少且差值最大的row_diff所在的行
    3. 重复直到删除了need_to_delete行
    """
    
    print("DELETING ROWS BY ROW_DIFF / 根据row_diff删除行:")
    #print(f"Need to delete: {need_to_delete} rows")
    print("="*70)
    
    # Get rows that have row_diff / 获取有row_diff的行
    rows_with_diff = df[df['row_diff'].notna()].copy()
    
    if len(rows_with_diff) == 0:
        print("❌ No rows with row_diff to delete")
        print("="*70 + "\n")
        return df
    
    #print(f"Total rows with row_diff: {len(rows_with_diff)}")
    
    # Track deleted indices / 追踪要删除的索引
    to_delete_indices = []
    deleted_count = 0
    
    # Loop until we've deleted enough rows / 循环直到删除足够的行
    while deleted_count < need_to_delete:
        # Get current row_diff distribution (excluding already marked for deletion)
        # 获取当前row_diff分布（排除已标记删除的）
        remaining = rows_with_diff[~rows_with_diff.index.isin(to_delete_indices)]
        
        if len(remaining) == 0:
            print(f"\n❌ No more rows to delete (deleted {deleted_count}/{need_to_delete})")
            break
        
        # Count distribution / 统计分布
        diff_counts = remaining['row_diff'].value_counts()
        
        #print(f"\nIteration {deleted_count + 1}:")
        #print(f"Current row_diff distribution / 当前row_diff分布:")
        #for diff_val, count in diff_counts.sort_index().items():
            #print(f"  row_diff={diff_val:3}: {count} items")
        
        # Find minimum count / 找到最小数量
        min_count = diff_counts.min()
        #print(f"\nMinimum count: {min_count}")
        

        # Get all row_diff values with minimum count / 获取所有最小数量的row_diff值
        min_count_diffs = diff_counts[diff_counts == min_count].index.tolist()
        #print(f"Row_diff values with min count: {min_count_diffs}")
        
        # Among these, find the one with largest absolute value / 其中找到绝对值最大的
        max_abs_diff = max(min_count_diffs, key=abs)
        #print(f"Largest absolute difference: {max_abs_diff}")
        
        # Get rows with this row_diff value / 获取这个row_diff值的行
        rows_to_delete = remaining[remaining['row_diff'] == max_abs_diff]
        
        #print(f"\nDeleting {len(rows_to_delete)} row(s) with row_diff={max_abs_diff}:")
        
        # Delete these rows (or as many as needed) / 删除这些行
        for idx in rows_to_delete.index:
            if deleted_count >= need_to_delete:
                break
            
            to_delete_indices.append(idx)
            deleted_count += 1
            
            row_no = df.loc[idx, 'row_No']
            text = df.loc[idx, 'text']
            mark = df.loc[idx, 'mark']
            
            #print(f"  - Index {idx}: '{text}' (row={row_no}, mark={mark})")
    
    # Actually delete from DataFrame / 真正从DataFrame删除
    df = df.drop(to_delete_indices).reset_index(drop=True)
    
    #print(f"✅ Deleted {deleted_count} rows")
    #print(f"Remaining rows: {len(df)}")
    #print("="*70 + "\n")
    
    return df

#====================================================================================================
def create_matched_dataframe(df):
    """
    Create final DataFrame with matched items and prices
    创建最终的配对DataFrame
    
    Columns: item, price
    Each row is a matched pair
    """
    
    print("CREATING MATCHED DATAFRAME / 创建配对DataFrame:")
    print("="*70)
    # Add row_match column / 增加一列row_match
    # 如果row_diff为none的直接用row_No的值，否则row_No加上row_diff
    df['row_match'] = df.apply(lambda row: row['row_No'] if pd.isna(row['row_diff']) else row['row_No'] + row['row_diff'], axis=1).astype(int)
    
    # Get unique row_match values / 获取唯一的row_match值
    unique_matches = df['row_match'].unique()
    unique_matches = sorted(unique_matches)
    
    #print(f"Total unique row_match values: {len(unique_matches)}")
    #print(f"唯一row_match值总数: {len(unique_matches)}\n")
    
    # Store matched pairs / 存储配对结果
    matched_list = []
    
    # Process each row_match value / 处理每个row_match值
    for match_value in unique_matches:
        # Get all rows with this row_match / 获取该row_match的所有行
        matched_rows = df[df['row_match'] == match_value]
        
        # Find text and price / 查找text和price
        texts = matched_rows[matched_rows['mark'] == 'text']
        prices = matched_rows[matched_rows['mark'] == 'price']
        
        # Only add if both text and price exist / 只有两者都存在时才添加
        if len(texts) > 0 and len(prices) > 0:
            # Get first text and first price / 取第一个text和第一个price
            item_text = texts.iloc[0]['text']
            price_text = prices.iloc[0]['text']
            
            # Add to list / 添加到列表
            matched_list.append({
                'item': item_text,
                'price': price_text
            })
            
            #print(f"✅ Matched (row_match={match_value}):")
            #print(f"     Item: {item_text}")
            #print(f"     Price: {price_text}")
        else:
            #print(f" ⚠️ Skipped (row_match={match_value}): ", end="")
            if len(texts) == 0:
                print(f"No text found")
            elif len(prices) == 0:
                print(f"No price found (has text: '{texts.iloc[0]['text']}')")
    
    # Create final DataFrame / 创建最终DataFrame
    final_df = pd.DataFrame(matched_list)
    
    print("FINAL MATCHED DATAFRAME / 最终配对DataFrame:")
    print(f"Total matched pairs: {len(final_df)}")
    print("="*70 + "\n")
    
    return final_df

# PART 4: Interactive Visualization
# 第四部分：交互式可视化

In [6]:

def create_interactive_viewer(receipt):
    """Interactive viewer with Add function / 带新增功能的交互式查看器"""
    
    items = receipt['items']
    
    if len(items) == 0:
        df = pd.DataFrame(columns=['Item', 'Price'])  # 创建空表格
        print("⚠️  No items detected, showing empty template")
    
    df = pd.DataFrame(items)
    original_df = df.copy()
    
    # Receipt ID
    if not hasattr(create_interactive_viewer, 'counter'):
        create_interactive_viewer.counter = {}
    
    date = receipt.get('date', 'Unknown')
    if date not in create_interactive_viewer.counter:
        create_interactive_viewer.counter[date] = 0
    create_interactive_viewer.counter[date] += 1
    receipt_id = create_interactive_viewer.counter[date]
    
    # ========== Calculate Total / 计算总额 ==========
    def calculate_total(dataframe):
        total = 0.0
        for idx, row in dataframe.iterrows():
            price_str = str(row['Price']).replace('$', '').strip()
            try:
                total += float(price_str)
            except:
                pass
        return total
    
    # ========== Display Header / 显示表头 ==========
    print("\n" + "="*70)
    print("               📋 RECEIPT VIEWER / 收据查看器")
    print("="*70)
    print(f"\n📅 Date / 日期: {receipt['date']}")
    print(f"🆔 Receipt ID / 收据编号: #{receipt_id}")
    print(f"📦 Total Items / 商品总数: {len(items)}")
    
    print("\n" + "-"*70)
    print("                    📝 ITEMS LIST / 商品列表")
    print("-"*70)
    
    # ========== Display Table / 显示表格 ==========
    table_output = widgets.Output()
    
    def refresh_table():
        """Refresh table display / 刷新表格显示"""
        display_df = df.copy()
        display_df.index = range(1, len(df) + 1)
        with table_output:
            clear_output()
            display(display_df)
    
    refresh_table()
    display(table_output)
    
    # ========== Dynamic Total / 动态总额 ==========
    total_output = widgets.Output()
    
    def update_total_display():
        new_total = calculate_total(df)
        with total_output:
            clear_output()
            print()
            print("="*70)
            print(f"{'💰 TOTAL / 总额:':>50} ${new_total:>8.2f}")
            print("="*70)
    
    update_total_display()
    display(total_output)
    
    # ========== Edit Section / 编辑区域 ==========
    print("\n" + "-"*70)
    print("                ✏️  EDIT DATA / 编辑数据")
    print("-"*70 + "\n")
    
    row_input = widgets.IntText(value=1, description='Row / 行号:', min=1)
    item_input = widgets.Text(value='', description='Item / 商品:', placeholder='Item name', layout=widgets.Layout(width='400px'))
    price_input = widgets.Text(value='', description='Price / 价格:', placeholder='0.00')
    
    add_btn = widgets.Button(description='✚ Add / 新增', button_style='success', icon='plus')
    update_btn = widgets.Button(description='✓ Update / 更新', button_style='info', icon='check')
    delete_btn = widgets.Button(description='✗ Delete / 删除', button_style='danger', icon='trash')
    reset_btn = widgets.Button(description='↺ Reset / 重置', button_style='warning', icon='refresh')
    save_btn = widgets.Button(description='💾 Export CSV / 导出', button_style='primary', icon='download')
    
    status_output = widgets.Output()
    
    # ========== Button Functions / 按钮功能 ==========
    
    def on_add(b):
        """Add new item / 新增商品"""
        nonlocal df
        with status_output:
            clear_output()
            
            # Check if item and price are provided
            if not item_input.value or not price_input.value:
                print("❌ Please enter both item and price!")
                print("❌ 请输入商品名和价格！")
                return
            
            # Create new row / 创建新行
            new_item = {
                'Item': item_input.value,
                'Price': price_input.value
            }
            
            # Add to DataFrame / 添加到DataFrame
            df.loc[len(df)] = new_item
            
            # Update row_input max value
            row_input.max = len(df)
            
            # Refresh display
            refresh_table()
            update_total_display()
            
            print(f"✅ New item added! / 新商品已添加！")
            print(f"   Item: {item_input.value}")
            print(f"   Price: {price_input.value}")
            print(f"   Total rows: {len(df)}")
            
            # Clear inputs / 清空输入框
            item_input.value = ''
            price_input.value = ''
    
    def on_update(b):
        """Update existing item / 更新现有商品"""
        nonlocal df
        with status_output:
            clear_output()
            row = row_input.value
            actual_idx = row - 1
            
            if 0 <= actual_idx < len(df):
                if item_input.value:
                    df.at[actual_idx, 'Item'] = item_input.value
                if price_input.value:
                    df.at[actual_idx, 'Price'] = price_input.value
                
                refresh_table()
                update_total_display()
                print(f"✅ Row {row} updated! / 第{row}行已更新！")
            else:
                print(f"❌ Row {row} does not exist! / 第{row}行不存在！")
                print(f"   Valid range: 1-{len(df)}")
    
    def on_delete(b):
        """Delete item / 删除商品"""
        nonlocal df
        with status_output:
            clear_output()
            row = row_input.value
            actual_idx = row - 1
            
            if 0 <= actual_idx < len(df):
                deleted_item = df.loc[actual_idx, 'Item']
                df = df.drop(actual_idx).reset_index(drop=True)
                
                row_input.max = len(df) if len(df) > 0 else 1
                
                refresh_table()
                update_total_display()
                print(f"✅ Row {row} deleted! / 第{row}行已删除！")
                print(f"   Deleted: {deleted_item}")
            else:
                print(f"❌ Row {row} does not exist!")
    
    def on_reset(b):
        """Reset to original / 重置为原始数据"""
        nonlocal df
        with status_output:
            clear_output()
            df = original_df.copy()
            
            row_input.max = len(df)
            
            refresh_table()
            update_total_display()
            print("✅ Data reset! / 数据已重置！")
    
    def on_save(b):
        """Export to CSV / 导出为CSV"""
        with status_output:
            clear_output()
            
            final_total = calculate_total(df)
            export_data = []
            
            for idx, row in df.iterrows():
                price_str = str(row['Price']).replace('$', '').strip()
                try:
                    cost = float(price_str)
                except:
                    cost = 0.0
                
                export_data.append({
                    'Date': receipt['date'],
                    'id': receipt_id,
                    'Item': row['Item'],
                    'cost': f"${cost:.2f}",
                    'total': f"${final_total:.2f}"
                })
            
            export_df = pd.DataFrame(export_data)
            date_str = receipt['date'].replace('/', '-')
            filename = f'receipt_{date_str}_id{receipt_id}.csv'
            export_df.to_csv(filename, index=False)
            
            print(f"✅ Exported to: {filename}")
            print(f"📊 Total: ${final_total:.2f}")
    
    # Connect buttons
    add_btn.on_click(on_add)
    update_btn.on_click(on_update)
    delete_btn.on_click(on_delete)
    reset_btn.on_click(on_reset)
    save_btn.on_click(on_save)
    
    # Display widgets
    display(row_input)
    display(item_input)
    display(price_input)
    display(widgets.HBox([add_btn, update_btn, delete_btn, reset_btn, save_btn]))
    display(status_output)
    
    print("\n" + "="*70)
    print("📖 Instructions / 使用说明:")
    print("-"*70)
    print("1. Add / 新增: Enter item and price, click Add")
    print("   输入商品名和价格，点击新增")
    print("2. Update / 更新: Enter row number and new values, click Update")
    print("   输入行号和新值，点击更新")
    print("3. Delete / 删除: Enter row number, click Delete")
    print("   输入行号，点击删除")
    print("4. Reset / 重置: Restore original data")
    print("   恢复原始数据")
    print("5. Export CSV / 导出: Save to file")
    print("   保存为文件")
    print("="*70)


# PART 5: Main Processing Function
# 第五部分：主处理函数

In [7]:

def process1_getdata(image_path, show_viewer=True):
    """
    Main function to process a receipt
    主函数，用于处理收据
    
    Args:
        image_path: path to receipt image / 收据图像路径
        show_viewer: whether to show interactive viewer / 是否显示交互式查看器
    """
    print("\n" + "="*70)
    print(f"Processing: {os.path.basename(image_path)}")
    print(f"处理: {os.path.basename(image_path)}")
    print("="*70)
    
    # Preprocess / 预处理
    print("Step 1: Preprocessing... / 步骤1：预处理...")
    preprocessed = preprocess_image(image_path)
    if preprocessed is None:
        print("❌ Preprocessing failed!")
        return None
    print("✅ Preprocessing done")
    
    # OCR / OCR识别
    print("\nStep 2: OCR Recognition... / 步骤2：OCR识别...")
    ocr = initialize_ocr()
    ocr_result = perform_ocr(ocr, image_path)
    #print("ocr_result:", ocr_result)
    #print(f"Keys in result: {list(ocr_result.keys())}")

    if len(ocr_result) == 0:
        print("❌ No text detected!")
        return None
    print(f"✅ Detected {len(ocr_result)} text regions")

    # Organize / 整理
    print("\nStep 3: Organizing data... / 步骤3：整理数据...")
    df_ocr_result = organize_ocr_results(ocr_result)
    print("✅ Data organized")

    return df_ocr_result

def process2_get_main_information(df_ocr_result):
    print("\nStep 4: Extracting information... / 步骤4：提取信息...")
    print("ADDING MARK COLUMN / 添加标记分类:")
    print("="*70)  

    #粗略标记文本分类
    df_ocr_result['mark'] = df_ocr_result['text'].apply(classify_text_type)

    #获取日期
    date_rows = df_ocr_result[df_ocr_result['mark'] == 'date']
    if len(date_rows) > 0:
        date_str = date_rows['text'].iloc[0]
    else:
        date_str = "Date not found"
    
    #增加行号
    df_mark_data =  mark_row_number(df_ocr_result)

    #识别有效信息表格部分，权重设置为1
    df_mark_data = add_priority_column(df_mark_data)
    df_valid_data = df_ocr_result[df_mark_data['priority'] == 1].copy()

    #增加列号
    df_valid_data = add_column_number(df_valid_data)

    #找出“单位”的文本
    df_valid_data = mark_units(df_valid_data)

    #进一步识别有效信息，权重设置为2
    df_valid_data = update_priority_marks(df_valid_data)

    #计算目标信息的行差值，用作后续的信息匹配
    df_goal_data,need_to_delete = calculate_row_diff(df_valid_data)
    
    #删除匹配不到的多余信息
    df_goal_data = delete_by_row_diff(df_goal_data, need_to_delete)

    #获取匹配好的信息
    df_matched_goal_data = create_matched_dataframe(df_goal_data)

    print("✅ Target information extraction completed")    
    return date_str, df_mark_data, df_valid_data, df_goal_data, df_matched_goal_data

    

def process3_display(df):
    if show_viewer and len(items) > 0:
        print("\nStep 5: Creating viewer... / 步骤5：创建查看器...")
        create_interactive_viewer(receipt_info)
    return df



# Entry point / 代码运行入口⬇️

In [8]:
# Step 1: OCR text extraction / 运行步骤1：图像识别提取文字


# Input image path / 输入图片路径
#image_path = 'SROIE2019/train/img/X00016469612.jpg'
image_path = 'SROIE2019/train/img/X00016469672.jpg'
#image_path = 'SROIE2019/train/img/X00016469622.jpg'
#image_path = 'SROIE2019/train/img/X51005361883.jpg' #一跑就崩

# Get raw OCR data and save to DataFrame / 获取识别后的原始数据并保存在 DataFrame
df_ocr_result = process1_getdata(image_path)
df_ocr_result


Processing: X00016469672.jpg
处理: X00016469672.jpg
Step 1: Preprocessing... / 步骤1：预处理...
✅ Preprocessing done

Step 2: OCR Recognition... / 步骤2：OCR识别...


Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/chiara/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/chiara/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/chiara/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/chiara/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('en_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/chiara/.paddlex/offic

✅ Detected 15 text regions

Step 3: Organizing data... / 步骤3：整理数据...
ORGANIZING OCR DATA / 整理OCR数据:
✅ Data organized


,text,confidence,left_top_x,left_top_y,right_top_x,right_top_y,left_bottom_x,left_bottom_y,right_bottom_x,right_bottom_y,center_x,center_y
0,tan chay yee,0.991900,97,0,316,0,316,33,97,33,206.5,16.5
1,SOON HUAT MACHINERY ENTERPRISE,0.984482,19,53,431,61,430,88,18,80,224.5,70.5
2,(JM0352019-K),0.974050,160,79,294,82,293,110,159,107,226.5,94.5
3,NO.53 JALAN PUTRA 1.,0.971054,127,107,327,110,326,135,126,131,226.5,121.0
4,TAMAN SRI PUTRA.,0.994149,142,133,312,135,312,159,142,157,227.0,146.0
...,...,...,...,...,...,...,...,...,...,...,...,...
86,327.00,0.999899,354,1082,413,1082,413,1103,354,1103,383.5,1092.5
87,CASH :,0.990495,244,1113,323,1110,324,1135,244,1138,284.0,1124.0
88,327.00,0.999916,355,1114,414,1114,414,1134,355,1134,384.5,1124.0
89,Change :,0.991664,226,1139,322,1135,323,1160,227,1163,274.5,1149.5


In [9]:
# Step 2: Process data and extract key info / 运行步骤2：处理数据，提取关键信息
date, df_mark_data, df_valid_data, df_goal_data, df_matched_goal_data = process2_get_main_information(df_ocr_result)
df_matched_goal_data


Step 4: Extracting information... / 步骤4：提取信息...
ADDING MARK COLUMN / 添加标记分类:
ADDING ROW NUMBER COLUMN / 添加行号:
ADDING PRIORITY COLUMN / 添加有效信息权重列:
ADDING COLUMN NUMBERS / 添加列号:
MARKING UNITS / 标记单位:
UPDATING PRIORITY MARKS / 更新优先级标记:
CALCULATING ROW DIFFERENCES / 计算行差:
DELETING ROWS BY ROW_DIFF / 根据row_diff删除行:
CREATING MATCHED DATAFRAME / 创建配对DataFrame:
FINAL MATCHED DATAFRAME / 最终配对DataFrame:
Total matched pairs: 8

✅ Target information extraction completed


,item,price
0,REPAIR ENGINE POWER SPRAYER (1UNIT),80.00
1,GIANT 606 OVERFLOW ASSY,160.00
2,ENGINE OIL,17.00
3,GREASE FOR TOOLS 40ML (AKODA),10.00
4,EY20 PLUG CHAMPION,6.00
5,STARTER TALI,8.00
6,EY20 STARTER HANDLE,10.00
7,HD40 1L COTIN,36.00


In [10]:
# Step 3: Visualize results and enable edit/export / 运行步骤3：结果可视化并提供修改导出功能

# Convert final data to dictionary / 将得到的最终数据转换为字典类型
receipt = {
    'date': date,
    'items': df_matched_goal_data.rename(columns={'item': 'Item', 'price': 'Price'}).to_dict('records')
}

# Pass data to frontend for display / 将数据传入前端进行显示
create_interactive_viewer(receipt)


               📋 RECEIPT VIEWER / 收据查看器

📅 Date / 日期: Date: 11/01/2019
🆔 Receipt ID / 收据编号: #1
📦 Total Items / 商品总数: 8

----------------------------------------------------------------------
                    📝 ITEMS LIST / 商品列表
----------------------------------------------------------------------


Output()

Output()


----------------------------------------------------------------------
                ✏️  EDIT DATA / 编辑数据
----------------------------------------------------------------------



IntText(value=1, description='Row / 行号:')

Text(value='', description='Item / 商品:', layout=Layout(width='400px'), placeholder='Item name')

Text(value='', description='Price / 价格:', placeholder='0.00')

Output()


📖 Instructions / 使用说明:
----------------------------------------------------------------------
1. Add / 新增: Enter item and price, click Add
   输入商品名和价格，点击新增
2. Update / 更新: Enter row number and new values, click Update
   输入行号和新值，点击更新
3. Delete / 删除: Enter row number, click Delete
   输入行号，点击删除
4. Reset / 重置: Restore original data
   恢复原始数据
5. Export CSV / 导出: Save to file
   保存为文件
